Imports

In [1]:
import sys
sys.path.append("../src")

In [2]:
import pandas as pd
from data.wav_data_reader import WavDataReader
from data.signal_dataset import SignalDataset
from features.wav_feature_extractor import WavFeatureExtractor
from features.features_dataset import FeaturesDataset
import data.preparation_eurythmy_data as ped
from models.fully_connected_classifier import FullyConnectedClassifier
from evaluation.hyperparameter_tuner import HyperparameterTuner

Reader

In [3]:
test_folder= r"..\data\interim\testing"

In [4]:
reader= WavDataReader(folder= test_folder)
signals, ids= reader.get_values_and_keys()
meas_df= ped.return_meas_labels_by_keys(ids)

Signal Dataset

In [5]:
#Initialize
signal_dataset= SignalDataset(signals= signals, features= meas_df)

#Process Data
signal_dataset.standardize_signals("zscore")
signal_dataset.segment_signals(segment_duration=1)
signal_dataset.remove_constant_signals()

#Add eurythmy letter data to the features
signal_dataset.features= ped.add_meas_letters(signal_dataset.features)

In [9]:
for rq_number in range(1, 5):  # Looping over RQ numbers 1, 2, 3, and 4
    try:
        indexes, targets = ped.get_indexes_and_targets_by_rq(rq_number, signal_dataset.features)
        print(f"Results for RQ{rq_number}:")
        print("Indexes:", indexes)
        print("Targets/Classes/Characters:", targets)
        print("------")  # Separator for readability
    except ValueError as e:
        print(e)

Results for RQ1:
Indexes: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 27

In [ ]:
klk

In [ ]:
signals, eurythmy_features= signal_dataset.get_data()

Feature Extractor

In [ ]:
%%time
extractor= WavFeatureExtractor()
all_features, feature_labels= extractor.extract_features_multiple_waveforms(waveforms= signals)

In [ ]:
feature_labels= feature_labels.append(eurythmy_labels)
all_features.append(eurythmy_features)

Features Dataset

In [ ]:
feat_dataset= FeaturesDataset(features= all_features, feature_labels= feature_labels)
feat_dataset.save_to_csv("raw_dataset.csv")
feat_dataset.process_features()
feat_dataset.shape()

In [ ]:
num_feat= len(feat_dataset.get_labels())
feat_dataset.head()
feat_dataset.save_to_csv("processed_dataset.csv")

In [ ]:
train_loader, val_loader, test_loader = feat_dataset.split_dataset_in_loaders(test_size=0.3, val_size=0.5, random_state=42)

## Search

In [ ]:
input_size= num_feat
output_size= 2
num_epochs = 5

param_grid = {
    'learning_rate': [0.1, 0.01],
    'dense_units': [64, 128],
    'dense_layers': [1, 2],
    'dropout_rate': [0.25, 0.5]
}

tuner = HyperparameterTuner(FullyConnectedClassifier, param_grid, train_loader, val_loader, num_epochs, input_size, output_size)
best_params, all_results = tuner.tune()

In [ ]:
print("Best Hyperparameters: \n", best_params)

In [ ]:
all_results.head(15)

In [ ]:
klk

## Model

In [ ]:
num_epochs = 5  # Number of training epochs

In [ ]:
model = FullyConnectedClassifier(
    input_size=5,  # Example input size
    hidden_layers=[128, 64],  # Two hidden layers with 128 and 64 units respectively
    output_size=2,  # Example output size
    dropout_rate=0.5,
    learning_rate=0.001
)

In [ ]:
model.train_model(train_loader, val_loader, num_epochs)

In [ ]:
test_predictions = model.predict(test_loader)

In [ ]:
actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

In [ ]:
correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)
